# Association Rules: Market Basket Analysis

## Import Libraries

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Load Dataset

Dataset obtained from [here](https://archive.ics.uci.edu/ml/datasets/Online+Retail+II).

<font color="red">Load in your data. Make sure to also set `encoding="ISO-8859-1"` as one of the `read_csv` parameters.</font>

In [13]:
df = pd.read_excel('datasets/online_retail.xlsx')


In [14]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [15]:
df.shape

(525461, 8)

We have 8 features and 541,910 rows.  
Looking at the source, this data comes from a UK-based online retail store selling gift-ware.  
  
Feature information:
- InvoiceNo: Invoice number. Nominal. A 6-digit integral number uniquely assigned to each transaction. If this code starts with the letter 'c', it indicates a cancellation.
- StockCode: Product (item) code. Nominal. A 5-digit integral number uniquely assigned to each distinct product.
- Description: Product (item) name. Nominal.
- Quantity: The quantities of each product (item) per transaction. Numeric.
- InvoiceDate: Invice date and time. Numeric. The day and time when a transaction was generated.
- UnitPrice: Unit price. Numeric. Product price per unit in sterling (Â£).
- CustomerID: Customer number. Nominal. A 5-digit integral number uniquely assigned to each customer.
- Country: Country name. Nominal. The name of the country where a customer resides.

## Pre-processing

### Drop nulls

<font color="red">Drop nulls</font>

In [23]:
df_cleaned = df.dropna()

df_cleaned.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [25]:
df.shape

(525461, 8)

We have 406,840 rows left after dropping any row with null values.

### Filter to only include positive quantities

In [29]:
df['Quantity'].unique()

array([   12,    48,    24,    10,    18,     3,    16,     4,     2,
           6,     8,     1,    28,    30,    60,    32,    56,     9,
          25,    36,    20,   -12,    -6,    -4,   -24,    -3,    -2,
         576,   288,    27,   -96,    64,   160,  -240,  -192,     5,
          -1,    72,    50,    54,   -50,     7,   800,    11,    58,
         192,    96,   144,   100,   180,   216,    15,   240,    22,
         108,    33,    90,    81,   -81,   -48,    17,    -5,    14,
          13,    40,    -8,    -9,    21,   120,   -44,   230, -1043,
        -117,   200,   168,    19,   280,   128,  -150,   -18,   -23,
          80,   300,   450,    61,  -770,  -720,   480,    -7,    88,
          46,   -31,   -16,   954,  -954,  -200,   504,    45,   467,
       -1012,  4320,  5184,  4008,   -25,   600,   505,   500,    23,
          74,    47,   -89,   432,   252,    75,   184,  -600,  -504,
        -252,  -246,  -169,    31,    38,  -106,   -36,   360,   107,
          49,   -42,

It is possible for a customer to cancel their order, resulting in a negative quantity. We only want to look at items the customers have actually purchased, so we will remove any negatives (or cancelled orders). 

In [32]:
df = df[df['Quantity']>1] # this allows us to filter only rows where quantity > 1

In [34]:
df.shape

(367017, 8)

After filtering, we have a remaining 397,925 rows.

### Filter to only UK transcations

In [ ]:
df['Country'].value_counts()

<font color="red">Filter the dataframe to include only transactions from the United Kingdom</font>

In [38]:
df_uk = df_cleaned[df_cleaned['Country'] == 'United Kingdom']


In [40]:
df_uk.shape, df_uk['Country'].value_counts()

((379423, 8),
 Country
 United Kingdom    379423
 Name: count, dtype: int64)

Now we should have 282,944 rows left.

But, each row is an individual item in a given transaction. To work with Apriori algorithm, we want each row to be a transaction along with all the purchased items.

### One Hot Encoding of our items

In [45]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


We just want a table with each row being an individual invoice, and each column being the various items (0 if not purchased, 1 if purchased). The exact quantity does not matter since we have already filtered out those more than 0 (meaning it was purchased).

In [48]:
df_enc = df[['Invoice','Description']].pivot_table(index=['Invoice'], columns=['Description'], aggfunc=[len], fill_value=0)

TypeError: '<' not supported between instances of 'str' and 'int'

There are some instances of 2's and 3's, etc... instead of just 1's. So we will clean that as well.

In [ ]:
def encode(x):
    if x >= 1:
        return 1
    # else
    return 0

In [ ]:
df_enc = df_enc.applymap(encode)

In [ ]:
df_enc

Let's briefly check this with our first invoice: 536365  
The below shows the sum of different items purchased for each invoice.

In [ ]:
df_enc[df_enc == 1].sum(axis=1)

When checking below, we see that invoice 536365 had 7 different items.

In [ ]:
df.head(10)

### Filter out transactions with only one purchase

Before we run Apriori algorithm, we only want to perform this on transactions that had more than one purchase. When looking for assocation rules, we are looking for what purchased items are related to each other. If a transaction only has one purchase, this doesn't help us much. *So we will simply remove those transactions.* :)

In [64]:
df_enc = df_encoded[(df_encoded[df_encoded == 1]).sum(axis=1) > 1]
df_enc


NameError: name 'df_encoded' is not defined

Originally, we had 16,649 rows of transactions. After filtering, we now have 15,371 rows.

## Apriori Algorithm

We can use the `mlxtend` library to apply the Apriori Algorithm. We will first install the package and then load it in.  
  
If you are interested in seeing how this works, you can also view their Github [here](https://github.com/rasbt/mlxtend). The more specific area on Apriori is [here](https://github.com/rasbt/mlxtend/tree/master/mlxtend/frequent_patterns).

### Load our library

In [66]:
!pip install mlxtend

   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 1.4/1.4 MB 23.2 MB/s eta 0:00:00


In [67]:
from mlxtend.frequent_patterns import apriori, association_rules

### Find frequent itemsets

<font color="red">Before continuing, take a look at the documentation for our `apriori` method.</font>

In [74]:
apriori?

Signature:
apriori(
    df,
    min_support=0.5,
    use_colnames=False,
    max_len=None,
    verbose=0,
    low_memory=False,
)
Docstring:
Get frequent itemsets from a one-hot DataFrame

Parameters
-----------
df : pandas DataFrame
  pandas DataFrame the encoded format. Also supports
  DataFrames with sparse data; for more info, please
  see (https://pandas.pydata.org/pandas-docs/stable/
       user_guide/sparse.html#sparse-data-structures)

  Please note that the old pandas SparseDataFrame format
  is no longer supported in mlxtend >= 0.17.2.

  The allowed values are either 0/1 or True/False.
  For example,

```
         Apple  Bananas   Beer  Chicken   Milk   Rice
    0     True    False   True     True  False   True
    1     True    False   True    False  False   True
    2     True    False   True    False  False  False
    3     True     True  False    False  False  False
    4    False    False   True     True   True   True
    5    False    False   True    False   True   Tru

<font color="red">Create a variable called `freq_itemsets` to save our results from the `apriori` method call. In this call, set `df_enc` as our DataFrame, `min_support` to be `0.03`, and `use_colnames` to `True`.</font>

In [77]:
from mlxtend.frequent_patterns import apriori

freq_itemsets = apriori(df_encoded, min_support=0.03, use_colnames=True)

import ace_tools as tools; tools.display_dataframe_to_user(name="Frequent Itemsets", dataframe=freq_itemsets)


C:\Users\jandr\anaconda3\Lib\site-packages\executing\executing.py:713: DeprecationWarning: ast.Str is deprecated and will be removed in Python 3.14; use ast.Constant instead
  right=ast.Str(s=sentinel),
C:\Users\jandr\anaconda3\Lib\ast.py:587: DeprecationWarning: Attribute s is deprecated and will be removed in Python 3.14; use value instead
  return Constant(*args, **kwargs)
C:\Users\jandr\anaconda3\Lib\site-packages\executing\executing.py:713: DeprecationWarning: ast.Str is deprecated and will be removed in Python 3.14; use ast.Constant instead
  right=ast.Str(s=sentinel),
C:\Users\jandr\anaconda3\Lib\ast.py:587: DeprecationWarning: Attribute s is deprecated and will be removed in Python 3.14; use value instead
  return Constant(*args, **kwargs)


NameError: name 'df_encoded' is not defined

In [ ]:
freq_itemsets.sort_values('support', ascending=False)

### Find the association rules

<font color="red">Again, take a look at the documentation for our `association_rules` method before continuing.</font>

In [79]:
association_rules?

Signature:
association_rules(
    df: pandas.core.frame.DataFrame,
    num_itemsets: int,
    df_orig: Optional[pandas.core.frame.DataFrame] = None,
    null_values=False,
    metric='confidence',
    min_threshold=0.8,
    support_only=False,
    return_metrics: list = ['antecedent support', 'consequent support', 'support', 'confidence', 'lift', 'representativity', 'leverage', 'conviction', 'zhangs_metric', 'jaccard', 'certainty', 'kulczynski'],
) -> pandas.core.frame.DataFrame
Docstring:
Generates a DataFrame of association rules including the
metrics 'score', 'confidence', and 'lift'

Parameters
-----------
df : pandas DataFrame
  pandas DataFrame of frequent itemsets
  with columns ['support', 'itemsets']

df_orig : pandas DataFrame (default: None)
  DataFrame with original input data. Only provided when null_values exist

num_itemsets : int
  Number of transactions in original input data

null_values : bool (default: False)
  In case there are null values as NaNs in the original i

In [81]:
association_rules(freq_itemsets, metric="confidence", min_threshold=0.60, num_itemsets = len(df_enc)).sort_values("confidence", ascending=False)

C:\Users\jandr\anaconda3\Lib\site-packages\executing\executing.py:713: DeprecationWarning: ast.Str is deprecated and will be removed in Python 3.14; use ast.Constant instead
  right=ast.Str(s=sentinel),
C:\Users\jandr\anaconda3\Lib\ast.py:587: DeprecationWarning: Attribute s is deprecated and will be removed in Python 3.14; use value instead
  return Constant(*args, **kwargs)
C:\Users\jandr\anaconda3\Lib\site-packages\executing\executing.py:713: DeprecationWarning: ast.Str is deprecated and will be removed in Python 3.14; use ast.Constant instead
  right=ast.Str(s=sentinel),
C:\Users\jandr\anaconda3\Lib\ast.py:587: DeprecationWarning: Attribute s is deprecated and will be removed in Python 3.14; use value instead
  return Constant(*args, **kwargs)


NameError: name 'freq_itemsets' is not defined

With a minimum confidence threshold of 60%, this is what we have (above).  
If someone buys a *green regency teacup and saucer* they are likely to also buy a *roses regency teacup and saucer*.

<font color="red">What if we look at a confidence threshold of 20%? What are some other rules?</font>

In [127]:
from mlxtend.frequent_patterns import association_rules
rules = association_rules(freq_itemsets, metric="confidence", min_threshold=0.20)

rules_sorted = rules.sort_values("confidence", ascending=False)
import ace_tools as tools; tools.display_dataframe_to_user(name="Association Rules with Confidence >= 20%", dataframe=rules_sorted)


## Conclusions

<font color="red">Discuss, based on the association rules found above, what are some strategies this online store could employ to increase sales?</font>

To increase sales, the online store can bundle items that are often bought together, like matching teacups, and offer discounts to encourage customers to buy more. They can show product suggestions like "People who bought this also bought that" on the website and in emails. Special promotions, like "Buy one, get 20% off another," can attract more purchases. Grouping related items into easy-to-find collections and sending personalized follow-up emails can also help. Seasonal sales, like holiday bundles, and loyalty rewards for buying sets can make shopping more exciting. These strategies make it easier and more fun for customers to shop while increasing the store’s profits.